In [1]:
from sklearn.metrics import r2_score, mean_squared_error
import joblib  # save model
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from numpy import absolute
from numpy import std
from numpy import mean
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.datasets import make_regression
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV

In [2]:

file = r'dataset\features\EURUSD.csv'
df = pd.read_csv(file)
df.set_index('date', inplace=True, drop=True)

################################################
features = df[['open',
 'high',
 'low',
 'close',
 'WPC',
 'HA_open',
 'HA_high',
 'HA_low',
 'HA_close',
 'EMA_4',
 'EMA_8',
 'EMA_16',
 'EMA_32',
 'bb_bbm_4',
 'bb_bbh_4',
 'bb_bbl_4',
 'bb_bbm_8',
 'bb_bbh_8',
 'bb_bbl_8',
 'bb_bbm_16',
 'bb_bbh_16',
 'bb_bbl_16',
 'bb_bbm_32',
 'bb_bbh_32',
 'bb_bbl_32']].copy()
targets = df[['open_24', 'close_24']].copy()

################################################
targets = targets*10000
################################################

In [3]:
sc_X = StandardScaler()
sc_y = StandardScaler()
x = sc_X.fit_transform(features.values)
y = sc_y.fit_transform(targets.values)

input_train, input_test, output_train, output_test = train_test_split(
    x, y, test_size=0.05)
print("train shape : {:.0f}".format(
    input_train.shape[0]/24), "days || test shape : {:.0f}".format(input_test.shape[0]/24), "days")

train shape : 1405 days || test shape : 74 days


In [4]:
param = {
    'estimator__kernel' : ('poly','rbf'),
    'estimator__C' : np.arange(1,52,10),
    'estimator__degree' : np.arange(3,8,1),
    'estimator__coef0' :  [0.01,0.1,1,10],
    'estimator__gamma' : ('auto','scale',0.01,0.1,1,10),
    'estimator__epsilon' : (0.0001,0.001,0.01,0.1,1),
    'estimator__verbose' : [2]
}
param = {
    'estimator__kernel' : ['rbf'],
    'estimator__C' : [25],
    'estimator__degree' : [4],
    'estimator__coef0' :  [0.01],
    'estimator__gamma' : [0.1],
    'estimator__epsilon' : [0.0001],
    'estimator__verbose' : [2]
}
param

{'estimator__kernel': ['rbf'],
 'estimator__C': [25],
 'estimator__degree': [4],
 'estimator__coef0': [0.01],
 'estimator__gamma': [0.1],
 'estimator__epsilon': [0.0001],
 'estimator__verbose': [2]}

In [5]:
model = SVR()
Multi_SVR = MultiOutputRegressor(model)
Grids = GridSearchCV(Multi_SVR,param,cv=None)
Multi_SVR.get_params().keys()

dict_keys(['estimator__C', 'estimator__cache_size', 'estimator__coef0', 'estimator__degree', 'estimator__epsilon', 'estimator__gamma', 'estimator__kernel', 'estimator__max_iter', 'estimator__shrinking', 'estimator__tol', 'estimator__verbose', 'estimator', 'n_jobs'])

In [6]:
Grids.fit(input_train,output_train)

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

GridSearchCV(estimator=MultiOutputRegressor(estimator=SVR()),
             param_grid={'estimator__C': [25], 'estimator__coef0': [0.01],
                         'estimator__degree': [4],
                         'estimator__epsilon': [0.0001],
                         'estimator__gamma': [0.1],
                         'estimator__kernel': ['rbf'],
                         'estimator__verbose': [2]})

In [7]:
print("Best score : ",Grids.best_score_)
print("Best param : ",Grids.best_params_)

Best score :  0.9840735133642247
Best param :  {'estimator__C': 25, 'estimator__coef0': 0.01, 'estimator__degree': 4, 'estimator__epsilon': 0.0001, 'estimator__gamma': 0.1, 'estimator__kernel': 'rbf', 'estimator__verbose': 2}


In [8]:
Grids.cv_results_

{'mean_fit_time': array([117.83177805]),
 'std_fit_time': array([2.48554504]),
 'mean_score_time': array([10.69899435]),
 'std_score_time': array([0.11583369]),
 'param_estimator__C': masked_array(data=[25],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_estimator__coef0': masked_array(data=[0.01],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_estimator__degree': masked_array(data=[4],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_estimator__epsilon': masked_array(data=[0.0001],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_estimator__gamma': masked_array(data=[0.1],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_estimator__kernel': masked_array(data=['rbf'],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_estimator__verbose': masked_array(da